# Transit Over Point Target
## 0. Imports

In [ ]:
from surface import *
from source import *
from model import *

import numpy as np
from time import time as Time

## 1. Define surface

In [ ]:
# surface
surf = Surface(origin=(0, 0), dims=(101, 101), fs=100)
surf.gen_flat(-10)

## 2. Create sources along a transit

In [ ]:
# source
ss = []
n = 200
for x in np.linspace(0, 10000, n):
    source = Source(1e-9, 0.5e-6, (x, 5050, 25000))
    source.chirp(9e6, 1e6)
    ss.append(source)

## 3. Create models and simulate for each source location

In [ ]:
ms = []
st = Time()
for i, s in enumerate(ss):
    print(f"Simulating: {i+1}/{len(ss)} ({round(100*((i+1)/len(ss)), 1)}%)", end="     \r")
    model = Model(surf, s)
    model.set_target((5050, 5050, -200))    # set target location
    model.gen_raypaths()    # generate raypaths
    model.comp_dopplers()
    model.gen_timeseries(show=False)    # simulate
    ms.append(model)
print(f"\n\nTotal processing time: {round((Time() - st)/60)} minutes and {round((Time() - st) % 60,2)} seconds")

## 4. Combine into radargram

In [ ]:
st = 166.8e-6   # start
en = 175e-6  # end
N = int((en-st)/(1e-9))      # how many "range bins?"
print(f"{N} range bins")

# fast time dimension
ft = np.linspace(st, en, N)

# output radargram
rdrgrm = np.zeros((N, n)).astype(np.complex128)

# iterate through models and add to output radargram
for i, m in enumerate(ms):
    real = np.interp(ft, m.ts, np.real(m.data))
    imag = np.interp(ft, m.ts, np.imag(m.data))
    rdrgrm[:, i] = real + 1j * imag

In [ ]:
np.save("rdrgrm.npy", rdrgrm)

## 5. Show output

In [ ]:
import plotly.express as px

fig = px.imshow(np.real(rdrgrm), aspect="auto", color_continuous_scale='gray', width=800, height=600)
ticktext = ['{0:5.1f}'.format(val) for val in np.arange(st*1e6, en*1e6, 1)]
tickvals = np.linspace(0, N, len(ticktext))

fig.update_yaxes(
    tickvals=tickvals,
    ticktext=ticktext,
    title_text="Time (µs)"
)
fig.show()